In [30]:
!ls ../input/abide-cc200/cc200/

CC200


In [31]:
#options: cc200, dosenbach160, aal
p_ROI = "cc200"
p_fold = 10
p_center = "Stanford"
p_mode = "whole"
p_augmentation = True
p_Method = "ASD-DiagNet"

In [32]:
parameter_list = [p_ROI,p_fold,p_center,p_mode,p_augmentation,p_Method]
print("*****List of patameters****")
print("ROI atlas: ",p_ROI)
print("per Center or whole: ",p_mode)
if p_mode == 'percenter':
    print("Center's name: ",p_center)
print("Method's name: ",p_Method)
if p_Method == "ASD-DiagNet":
    print("Augmentation: ",p_augmentation)


*****List of patameters****
ROI atlas:  cc200
per Center or whole:  whole
Method's name:  ASD-DiagNet
Augmentation:  True


In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from functools import reduce
from sklearn.impute import SimpleImputer
import time
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import pyprind
import sys
import pickle
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import silhouette_samples, silhouette_score
import torch.optim as optim
from sklearn.metrics import confusion_matrix
from scipy import stats
from sklearn import tree
from sklearn.cluster import MiniBatchKMeans
import functools
import numpy.ma as ma # for masked arrays
import pyprind
import random
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from tqdm.notebook import tqdm
import wandb
from itertools import groupby
from sklearn.decomposition import PCA


# !wandb login d164742a4a99e4e581f543102aff0992153ad225

## Importing the data 

In [58]:
config = dict({
    "bs" : 64,
    "lr" : 1e-4,
    "epochs" : 100
})

In [35]:
def get_key(filename):
    f_split = filename.split('_')
    if f_split[3] == 'rois':
        key = '_'.join(f_split[0:3]) 
    else:
        key = '_'.join(f_split[0:2])
    return key

In [36]:
cc200_data_path = '../input/abide-cc200/cc200/CC200'#cc200'#path to time series data
flist = os.listdir(cc200_data_path)

fid = []
for f in flist:
    fid.append(get_key(f))

    
data_df = pd.read_csv('../input/abide-cc200/Phenotypic_V1_0b_preprocessed1.csv')#path 
data_df = data_df[data_df['FILE_ID'].isin(fid)]
data_df.DX_GROUP = data_df.DX_GROUP.map({1: 1, 2:0})
data_df['FILE_PATH'] = data_df['FILE_ID'].apply(lambda x : os.path.join(cc200_data_path,x + '_rois_cc200.1D')) 


# print(data_df.head())
print(len(data_df))
print(data_df['FILE_PATH'].values[0])

1035
../input/abide-cc200/cc200/CC200/Pitt_0050003_rois_cc200.1D


In [37]:
len(flist)

1035

### Helper functions for computing correlations

In [38]:
def get_label(filename):
    assert (filename in labels)
    return labels[filename]


def get_corr_data(path):
    
    df = pd.read_csv(path, sep='\t')
            
    with np.errstate(invalid="ignore"):
        corr = np.nan_to_num(np.corrcoef(df.T))
        mask = np.invert(np.tri(corr.shape[0], k=-1, dtype=bool))
        m = ma.masked_where(mask == 1, mask)
        return ma.masked_where(m, corr).compressed()


def get_corr_matrix(filename,data_path):
    # returns correlation matrix
    for file in os.listdir(data_path):
        if file.startswith(filename):
            df = pd.read_csv(os.path.join(data_path, file), sep='\t')
    with np.errstate(invalid="ignore"):
        corr = np.nan_to_num(np.corrcoef(df.T))
        return corr

def confusion(g_turth,predictions):
    tn, fp, fn, tp = confusion_matrix(g_turth,predictions).ravel()
    accuracy = (tp+tn)/(tp+fp+tn+fn)
    sensitivity = (tp)/(tp+fn)
    specificty = (tn)/(tn+fp)
    return accuracy,sensitivity,specificty

def get_regs(samplesnames,regnum):
    # returns region index array
    datas = []
    for sn in samplesnames:
        datas.append(all_corr[sn][0])
    datas = np.array(datas)     # datas shape len(samplesnames)x19900
    avg=[]
    for ie in range(datas.shape[1]):
        avg.append(np.mean(datas[:,ie]))
    avg=np.array(avg)
    highs=avg.argsort()[-regnum:][::-1]
    lows=avg.argsort()[:regnum][::-1]
    regions=np.concatenate((highs,lows),axis=0) # shape (9950,)
    return regions

In [39]:
labels = data_df['DX_GROUP'].values
fpaths = data_df['FILE_PATH']
sfc = []


for path in fpaths:
    corr_data = get_corr_data(path)
    sfc.append(corr_data)
    
sfc = np.asarray(sfc)
print(sfc.shape)
print(labels.shape)

(1035, 19900)
(1035,)


In [40]:
print(np.min(sfc))

print(np.max(sfc))

-0.9592505245533961
0.9900251023663668


In [41]:
pca = PCA(n_components=512)
sfc_pca = pca.fit_transform(sfc)

In [42]:
print(sfc_pca.shape)

(1035, 512)


## Defining dataset class

In [43]:
class ASDDataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y
        pass
    def __getitem__(self,idx):
        return torch.tensor(self.x[idx],dtype=torch.float),torch.tensor(self.y[idx],dtype=torch.float)
        pass
    def __len__(self):
        return len(self.x)
        pass
        

In [44]:
d = ASDDataset(sfc,labels)

d.__getitem__(0)

(tensor([ 0.2489,  0.1380,  0.1472,  ...,  0.0305,  0.1095, -0.2725]),
 tensor(1.))

In [65]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_1 = nn.Linear(512, 128)
        self.hidden_2 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)
    
        self.bn1 = nn.BatchNorm1d(num_features=128)
        self.bn2 = nn.BatchNorm1d(num_features=64)
        self.bn3 = nn.BatchNorm1d(num_features=64)
        self.bn4 = nn.BatchNorm1d(num_features=32)
        
        # Define sigmoid activation and softmax output 
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, x):
        # Pass the input tensor through each of our operations
        x = self.hidden_1(x)
        x = self.sigmoid(x)
#         x = self.dropout(x)
        
        x = self.hidden_2(x)
        x = self.sigmoid(x)
#         x = self.dropout(x)
            
        x = self.output(x)
        
        return x.squeeze()

In [68]:

def loss_fn(logits, targets):
  return nn.BCEWithLogitsLoss()(logits,targets)

def train_loop_fn(data_loader, model, optimizer, device, scheduler=None):
  model.train()
  train_loss = []
  final_targets = []
  final_predictions = []
  bar = tqdm(data_loader,leave=False)
  for (inputs, targets) in bar:
    inputs, targets = inputs.to(device), targets.to(device)
    optimizer.zero_grad()
    logits = model(inputs)
#     print(logits.shape)
#     print(targets.shape)
    loss = loss_fn(logits , targets)
    train_loss.append(loss.detach().cpu().numpy())
    
    proba = torch.sigmoid(logits).detach().cpu().numpy()
    predictions = np.ones_like(proba, dtype=np.int32)
    predictions[proba < 0.5] = 0
    final_targets.append(targets.detach().cpu().numpy())
    final_predictions.append(predictions)
    
    loss.backward()
    optimizer.step()

    if scheduler is not None:
      scheduler.step()

  final_targets = np.concatenate(final_targets)
  final_predictions = np.concatenate(final_predictions)
  train_accuracy = np.mean(final_targets == final_predictions)
  mlp_acc,mlp_sens,mlp_spef = confusion(final_targets,final_predictions)

    
  
  return np.mean(train_loss), mlp_acc

def eval_fn(data_loader, model, device):
    model.eval()
    eval_loss = []
    final_targets = []
    final_predictions = []
    bar = tqdm(data_loader,leave=False)
    
    with torch.no_grad():
      for (inputs, targets) in bar:
        inputs, targets = inputs.to(device), targets.to(device)
        logits = model(inputs)
        loss = loss_fn(logits , targets)
        eval_loss.append(loss.detach().cpu().numpy())
        
        proba = torch.sigmoid(logits).detach().cpu().numpy()
        predictions = np.ones_like(proba, dtype=np.int32)
        predictions[proba < 0.5] = 0
        
        final_predictions.append(predictions)
        final_targets.append(targets.detach().cpu().numpy())
        
    final_targets = np.concatenate(final_targets)
    final_predictions = np.concatenate(final_predictions)
    mlp_acc,mlp_sens,mlp_spef = confusion(final_targets,final_predictions)
    metrics_dict = {'accuracy': np.round(mlp_acc, 4), 
                        'senstivity' : np.round(mlp_sens,4), 
                        'specificity' : np.round(mlp_spef,4), 
                        'loss' : np.round(np.mean(eval_loss),4)}

    accuracy = np.mean(final_targets == final_predictions)
    eval_loss = np.mean(eval_loss)

    return accuracy, metrics_dict

def run_k_fold(X,Y,num_splits=10):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  device = "cpu"
  data_dir = '../input/abideho/abide_ho/HO'

  if num_splits is not None:
    # list to store metrics after each fold
    crossval_acc=[]
    crossval_sen=[]
    crossval_spec=[]
    crossval_loss=[]
    skf = StratifiedKFold(n_splits=num_splits, random_state=1, shuffle=True) 
    kk = 0
    for train_index, test_index in skf.split(X, Y):

        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]


        verbose = (True if (kk == 0) else False)

        train_dataset = ASDDataset(x_train,y_train)
        test_dataset = ASDDataset(x_test,y_test)

        train_dataloader = DataLoader(train_dataset,batch_size=config["bs"],shuffle=True)
        test_dataloader = DataLoader(test_dataset,batch_size=config["bs"],shuffle=False)
        
#         model = CTransformer(1378, 12, 4, 4, 4, 1,device)
        model = Network()
#         model = LSTM()
        model.to(device)
        optimizer = optim.Adam(model.parameters(),lr=config["lr"],weight_decay=0.01)

        for epoch in range(config["epochs"]):

            
            train_loss, train_accuracy = train_loop_fn(train_dataloader,model,optimizer,device)                                                   
            print(f'Epoch {epoch + 1}/{config["epochs"]}')

            content = f'Classifier Train loss: {(train_loss):.4f}, Train Acc: {(train_accuracy):.4f}'
            print(content)

        accuracy, metrics_dict = eval_fn(test_dataloader, model, device)
        print("-----------------------------")
        print(f'Fold {kk+1}/{num_splits}')
        content = f'{metrics_dict}'
        print(content)
        print("-----------------------------")

        crossval_acc.append(metrics_dict['accuracy'])
        crossval_sen.append(metrics_dict['senstivity'])
        crossval_spec.append(metrics_dict['specificity'])
        crossval_loss.append(metrics_dict['loss'])

        #save the model after each fold
#         recorder = {'optimizer': optimizer.state_dict(),
#         'model': model.state_dict(),
#         'fold' : kk+1}

#         torch.save(recorder, f'{NAME}.pt')
        
        kk = kk + 1

    print("*********************************")    
    print(f'Average Value after 10 Folds------->')
    content = f'Accuracy: {np.round(np.mean(crossval_acc),4)}, Senstivity: {np.round(np.mean(crossval_sen),4)}, Specificity: {np.round(np.mean(crossval_spec),4)}, Loss: {np.round(np.mean(crossval_loss),4)}'
    print(content)
    print("*********************************")

In [69]:
run_k_fold(sfc_pca,labels,num_splits=10)

  0%|          | 0/15 [00:00<?, ?it/s]

Epoch 1/100
Classifier Train loss: 0.6957, Train Acc: 0.4876


  0%|          | 0/15 [00:00<?, ?it/s]

Epoch 2/100
Classifier Train loss: 0.6938, Train Acc: 0.4876


  0%|          | 0/15 [00:00<?, ?it/s]

...

Epoch 99/100
Classifier Train loss: 0.4244, Train Acc: 0.8680


  0%|          | 0/15 [00:00<?, ?it/s]

Epoch 100/100
Classifier Train loss: 0.4233, Train Acc: 0.8734


  0%|          | 0/2 [00:00<?, ?it/s]

-----------------------------
Fold 10/10
{'accuracy': 0.7476, 'senstivity': 0.74, 'specificity': 0.7547, 'loss': 0.5609}
-----------------------------
*********************************
Average Value after 10 Folds------->
Accuracy: 0.6947, Senstivity: 0.6754, Specificity: 0.7132, Loss: 0.5787000060081482
*********************************


In [ ]:
Average Value after 10 Folds------->
Accuracy: 0.6773, Senstivity: 0.6577, Specificity: 0.6962, Loss: 0.7893000245094299
                
40 epochs                
Average Value after 10 Folds------->
Accuracy: 0.6927, Senstivity: 0.6654, Specificity: 0.7188, Loss: 0.5934000015258789
                
                
100 epochs with 0.01 weight decay
Average Value after 10 Folds------->
Accuracy: 0.6985, Senstivity: 0.6813, Specificity: 0.7151, Loss: 0.580299973487854
                
Average Value after 10 Folds------->
Accuracy: 0.7004
                
                
80 epochs with 0.01 weight decay              
Average Value after 10 Folds------->
Accuracy: 0.6966, Senstivity: 0.6713, Specificity: 0.7207, Loss: 0.5986999869346619
                
100 epochs with 0.02 weight decay              
Average Value after 10 Folds------->
Accuracy: 0.6522, Senstivity: 0.5113, Specificity: 0.7868, Loss: 0.6704000234603882
                
  
100 epochs with 0.01 weight decay and dropout
Average Value after 10 Folds------->
Accuracy: 0.6937, Senstivity: 0.6634, Specificity: 0.7226, Loss: 0.5946999788284302
                
                